In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
import time
import os
import copy
import csv
import pandas as pd
import random

from PIL import Image

# 대화 모드
plt.ion()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True


cuda:0


In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5]),
])

# dataset 및 dataloader
target_dir = "dataset/train"
dataset = torchvision.datasets.ImageFolder(root=target_dir,
                                           transform=transform)
class_names = dataset.classes

dataset_sizes = {}
dataset_sizes["train"] = int(0.8 * len(dataset))
dataset_sizes["val"] = len(dataset) - dataset_sizes["train"]

datasets = {}
datasets["train"], datasets["val"] = torch.utils.data.random_split(
    dataset, [dataset_sizes["train"], dataset_sizes["val"]])

dataloaders = {}
dataloaders["train"] = torch.utils.data.DataLoader(datasets["train"],
                                                   batch_size=32,
                                                   shuffle=True,
                                                   num_workers=8)
dataloaders["val"] = torch.utils.data.DataLoader(datasets["val"],
                                                 batch_size=16,
                                                 shuffle=True,
                                                 num_workers=2)

for x in ["train", "val"]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))

print("Classes: ")
print(class_names)


Loaded 7064 images under train
Loaded 1766 images under val
Classes: 
['adult', 'cigaret', 'gun']


In [ ]:

dataset_sizes = {}
dataset_sizes["train"] = int(0.8 * len(dataset))
dataset_sizes["val"] = len(dataset) - dataset_sizes["train"]

datasets = {}
datasets["train"], datasets["val"] = torch.utils.data.random_split(
    dataset, [dataset_sizes["train"], dataset_sizes["val"]])

dataloaders = {}
dataloaders["train"] = torch.utils.data.DataLoader(datasets["train"],
                                                   batch_size=32,
                                                   shuffle=True,
                                                   num_workers=8)
dataloaders["val"] = torch.utils.data.DataLoader(datasets["val"],
                                                 batch_size=16,
                                                 shuffle=True,
                                                 num_workers=2)

for x in ["train", "val"]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))

print("Classes: ")
print(class_names)

In [21]:
# def imshow(inp, title=None):
#     inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([.5, .5, .5])
#     std = np.array([.5, .5, .5])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)

# def show_databatch(inputs, classes):
#     out = torchvision.utils.make_grid(inputs)
#     imshow(out, title=[class_names[x] for x in classes])

# # 학습 데이터의 배치
# inputs, classes = next(iter(dataloaders['val']))

# # 배치로부터 격자 형태의 이미지
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

In [3]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval() # 평가 모드; batch norm, dropout, ... X
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'ground truth:{class_names[labels[j]]}, predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [4]:
def eval_model(model, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    
    for i, data in enumerate(dataloaders["val"]):
        model.eval()
        inputs, labels = data[0].to(device), data[1].to(device)

        outputs = model(inputs)

        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)

        loss_test += loss.item()
        acc_test += torch.sum(preds == labels.data)
        
    avg_loss = loss_test / dataset_sizes["val"]
    avg_acc = acc_test.double() / dataset_sizes["val"]
    
    elapsed_time = time.time() - since
    print()
    print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Avg loss (test): {:.4f}".format(avg_loss))
    print("Avg acc (test): {:.4f}".format(avg_acc))

In [5]:
class VGG11(nn.Module):
    def __init__(
        self,
        num_classes: int = 1000,
    ) -> None:
        super(VGG11, self).__init__()
        """
        특징 추출 레이어
        """
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        """
        Conv + Fully Conn 구성 대비,
        Conv + AdaptiveAvgPooling + Fully Connected 구성이 다음과 같은 장점을 가짐:
        1. 다양한 입력 이미지 사이즈
        2. 학습 파라미터 수 감소
        """
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        """
        앞선 conv layer에서 특징을 추출했다면,
        추출된 특징들(512개)을 이용해 분류
        """
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [6]:
# 모델 생성 및 사전훈련된 가중치 불러오기
model = VGG11()
# WEIGHTS_URL = "https://download.pytorch.org/models/vgg11-bbd30ac9.pth"
# model.load_state_dict(torch.hub.load_state_dict_from_url(WEIGHTS_URL, progress=True))
model.load_state_dict(torch.load("vgg11-8a719046.pth"))
print(model.classifier[6].out_features) # 1000 


# 이전 레이어 학습 정지
for param in model.features.parameters():
    param.require_grad = False

# 새로운 레이어 추가
num_features = model.classifier[6].in_features
features = list(model.classifier.children())[:-1]
features.extend([nn.Linear(num_features, len(class_names))])
model.classifier = nn.Sequential(*features) # classifier 변경
print(model)

1000
VGG11(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=F

In [7]:
model.to(device)
    
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
print("Test before training")
eval_model(model, criterion)
visualize_model(model)

Test before training

Evaluation completed in 0m 11s
Avg loss (test): 0.1368
Avg acc (test): 0.4100


In [8]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        
        # training
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        
        model.train(True)
        
        # 데이터 반복
        for i, data in enumerate(dataloaders["train"]):
            inputs, labels = data[0].to(device), data[1].to(device)
            
            # 파라미터 경사도 초기화
            optimizer.zero_grad()
            
            outputs = model(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            # 역전파
            loss.backward()
            # 최적화
            optimizer.step()
            
            loss_train += loss.item()
            acc_train += torch.sum(preds == labels.data)
        
        avg_loss = loss_train / dataset_sizes["train"]
        avg_acc = acc_train.double() / dataset_sizes["train"]
        
        # validation
        model.train(False)
        model.eval()
            
        for i, data in enumerate(dataloaders["val"]):
            inputs, labels = data[0].to(device), data[1].to(device)

            outputs = model(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss_val += loss.item()
            acc_val += torch.sum(preds == labels.data)
        
        avg_loss_val = loss_val / dataset_sizes["val"]
        avg_acc_val = acc_val.double() / dataset_sizes["val"]
        
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        print('-' * 10)
        print()
        
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(model.state_dict())
        
    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model.to(device)
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=30)
# 모델 저장
torch.save(model.state_dict(), 'model.pt')

Epoch 0/30
----------
Epoch 0 result: 
Avg loss (train): 0.0040
Avg acc (train): 0.9534
Avg loss (val): 0.0048
Avg acc (val): 0.9711
----------

Epoch 1/30
----------
Epoch 1 result: 
Avg loss (train): 0.0011
Avg acc (train): 0.9894
Avg loss (val): 0.0027
Avg acc (val): 0.9887
----------

Epoch 2/30
----------
Epoch 2 result: 
Avg loss (train): 0.0005
Avg acc (train): 0.9948
Avg loss (val): 0.0027
Avg acc (val): 0.9887
----------

Epoch 3/30
----------
Epoch 3 result: 
Avg loss (train): 0.0003
Avg acc (train): 0.9967
Avg loss (val): 0.0025
Avg acc (val): 0.9881
----------

Epoch 4/30
----------
Epoch 4 result: 
Avg loss (train): 0.0001
Avg acc (train): 0.9992
Avg loss (val): 0.0032
Avg acc (val): 0.9887
----------

Epoch 5/30
----------
Epoch 5 result: 
Avg loss (train): 0.0001
Avg acc (train): 0.9996
Avg loss (val): 0.0029
Avg acc (val): 0.9881
----------

Epoch 6/30
----------
Epoch 6 result: 
Avg loss (train): 0.0001
Avg acc (train): 0.9990
Avg loss (val): 0.0028
Avg acc (val): 0.98

In [10]:
eval_model(model, criterion)


Evaluation completed in 0m 10s
Avg loss (test): 0.0029
Avg acc (test): 0.9938


In [11]:
model = VGG11(len(class_names))
model.load_state_dict(torch.load("model.pt"))
model.to(device)
model.eval()

VGG11(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [12]:
paths = []
target_dir = "dataset/test_images"
for root, _, fnames in sorted(os.walk(target_dir, followlinks=True)):
    for fname in sorted(fnames):
        path = os.path.join(target_dir, fname)
        paths += [path]

In [13]:
predictions = []

batch_size = 128
for i in range(round((len(paths) / batch_size) + 0.5)):
    # 이미지 batch 처리
    batch = paths[i*batch_size:(i+1)*batch_size]
    inputs = []
    for j, path in enumerate(batch):
        img = Image.open(path).convert('RGB')
        img = transform(img)
        inputs.append(img)

    inputs = torch.stack(inputs).to(device)

    # prediction
    preds = model(inputs)
    predictions += [preds.detach().cpu().numpy()]
    
    del preds
    torch.cuda.empty_cache()

In [14]:
with open('submission.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(["Id","Category"])
    for i, batch in enumerate(predictions):
        for j, pred in enumerate(batch):
            row = [os.path.basename(paths[i * batch_size + j]), class_names[pred.argmax()]]
            writer.writerow(row)

In [15]:
for i, batch in enumerate(predictions):
    for j, pred in enumerate(batch):
        row = [os.path.basename(paths[i * batch_size + j]), class_names[pred.argmax()]]
        print(row)

['1.jpg', 'cigaret']
['10.jpeg', 'gun']
['11.jpg', 'gun']
['12.jpg', 'gun']
['13.jpg', 'gun']
['14.jpg', 'gun']
['15.jpg', 'gun']
['16.jpg', 'gun']
['17.jpg', 'gun']
['18.jpg', 'gun']
['19.jpg', 'gun']
['2.jpg', 'cigaret']
['20.jpg', 'gun']
['21.jpg', 'gun']
['22.jpg', 'gun']
['23.jpg', 'gun']
['25.jpg', 'adult']
['26.jpg', 'adult']
['3.jpg', 'cigaret']
['4.jpg', 'gun']
['5.jpg', 'cigaret']
['6.jpg', 'adult']
['7.jpg', 'gun']
['8.jpg', 'adult']
['9.jpg', 'cigaret']
['person.jpg', 'adult']
['person2.jpg', 'adult']
['person3.jpg', 'adult']
['prefix_reddit_sub_nsfwfashion_-Marilh_a_Peillard_by_Fred_Meylan-K52bsk7.jpg.jpeg', 'adult']
['prefix_reddit_sub_nsfwfashion_hurch__x-post_r_fashionextramile_-IZEP3zh.jpg.jpeg', 'adult']
['prefix_reddit_sub_nsfwfashion_ie_Huntington_Whiteley_-__Angels__2018-vfo9511.png', 'adult']
['prefix_reddit_sub_nsfwfashion_m_NuMuses__HQ__processed_in_Photoshop_-ha4hcuL.png', 'adult']
['prefix_reddit_sub_nsfwfashion_ook_full_frontal_-_Playboy_Mexico-cqgybd2.jpg.jp